In [1]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # 학습, 테스트set 구분
from sklearn.tree import export_graphviz # tree 시각화를 위해
import graphviz # tree 시각화
from sklearn.metrics import f1_score# 성능지표를 계산하기 위해 import
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/LGaimers/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/LGaimers/test.csv')

In [3]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class','Y_Quality'])
train_quality = train_df['Y_Quality']
train_class = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [4]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,T100306,T_31,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,T100304,O_31,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#전처리

In [5]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [6]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [7]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.34,40.89,32.56,34.09,77.77,0.0,0.0,0.0,0.0,0.0
1,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.89,42.82,43.92,35.34,72.55,0.0,0.0,0.0,0.0,0.0
2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.19,36.65,42.47,36.53,78.35,0.0,0.0,0.0,0.0,0.0
3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.74,39.17,52.17,30.58,71.78,0.0,0.0,0.0,0.0,0.0
4,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.70,41.89,46.93,33.09,76.97,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
594,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.47,53.07,50.89,55.10,66.49,1.0,0.0,0.0,0.0,0.0
595,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,1.0,0.0,0.0,0.0,0.0
596,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0


## Linear Regression : feature 생성

In [8]:
# # 학습, 테스트 데이터 분리 (0.7:0.3)
# seed = 2022
# train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.3, random_state=seed)

In [9]:
seed = 2022
train_x, valid_x, train_quality, valid_quality, train_class, valid_class = (
    train_x[:400], train_x[400:], train_quality[:400], train_quality[400:], train_class[:400], train_class[400:])

In [10]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression()
LR.fit(train_x, train_quality)

LinearRegression()

In [11]:
LR_pred = LR.predict(valid_x)
LR_pred

array([ 0.54116911,  0.53341552,  0.53970765,  0.53213369,  0.49555144,
        0.51800731,  0.54408341,  0.53336369,  0.53190969,  0.53392574,
        0.51364361,  0.5314139 ,  0.53234751,  0.52995752,  0.52758866,
        0.53006835,  0.53297671,  0.5256272 ,  0.50392532,  0.54478418,
        0.52869068,  0.52685756,  0.52092913,  0.52918082,  0.51226343,
        0.53450339,  0.54461996,  0.52363286,  0.53806411,  0.52357966,
        0.52773975,  0.5259942 ,  0.53240422,  0.50055386,  0.53110162,
        0.52000355,  0.49932086,  0.54544353,  0.53367621,  0.51701829,
        0.5419104 ,  0.53273577,  0.52191581,  0.5292353 ,  0.53009933,
        0.52809842,  0.52771733,  0.51989201,  0.53738497,  0.51180774,
        0.52376216,  0.5314013 ,  0.52583553,  0.51356592,  0.54020675,
        0.5267286 ,  0.52186678,  0.53536454,  0.52540109,  0.53454501,
        0.53540293,  0.53645899,  0.52708811,  0.52544658,  0.53276964,
        0.53633957,  0.52170211,  0.52482709,  0.52327359,  0.52

In [12]:
# LR_class = []

# for i in LR_pred:
#   if i<=0.52507: LR_class.append(0)
#   elif 0.52507<i<0.5349: LR_class.append(1)
#   else: LR_class.append(2)

# np.array(LR_class)

In [13]:
# LR_f1 = f1_score(valid_class, LR_class, average='micro')
# LR_f1

# DecisionTree 학습

In [14]:
DT_train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class'])
DT_train_y = train_df['Y_Class']

DT_test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [15]:
DT_train_x = DT_train_x.fillna(0)
DT_test_x = DT_test_x.fillna(0)

In [16]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(DT_train_x[i])
    DT_train_x[i] = le.transform(DT_train_x[i])
    
    for label in np.unique(DT_test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    DT_test_x[i] = le.transform(DT_test_x[i]) 
print('Done.')

Done.


In [17]:
# 학습, 테스트 데이터 분리 
seed = 2022
DT_train_x, DT_valid_x, DT_train_y, DT_valid_y = DT_train_x[:400], DT_train_x[400:], DT_train_y[:400], DT_train_y[400:]

In [18]:
DT_train_x

,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,0.533433,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.34,40.89,32.56,34.09,77.77,0.0,0.0,0.0,0.0,0.0
1,0.541819,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.89,42.82,43.92,35.34,72.55,0.0,0.0,0.0,0.0,0.0
2,0.531267,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.19,36.65,42.47,36.53,78.35,0.0,0.0,0.0,0.0,0.0
3,0.537325,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.74,39.17,52.17,30.58,71.78,0.0,0.0,0.0,0.0,0.0
4,0.531590,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.70,41.89,46.93,33.09,76.97,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.532863,5,2,2.0,91.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
396,0.526921,5,2,2.0,96.0,0.0,45.0,10.0,0.0,51.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
397,0.531414,4,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
398,0.528348,4,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0


In [19]:
DT_valid_x

,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
400,0.531684,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
401,0.529641,4,2,1.0,94.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
402,0.526705,5,2,2.0,88.0,0.0,45.0,10.0,0.0,54.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
403,0.530411,5,2,2.0,93.0,0.0,45.0,10.0,0.0,53.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
404,0.500856,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,0.526546,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
594,0.524022,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.47,53.07,50.89,55.1,66.49,1.0,0.0,0.0,0.0,0.0
595,0.521289,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,1.0,0.0,0.0,0.0,0.0
596,0.531375,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [20]:
DT = DecisionTreeClassifier(random_state=seed)
DT.fit(DT_train_x, DT_train_y)

DecisionTreeClassifier(random_state=2022)

In [21]:
valid_x.insert(0, 'Y_Quality',LR_pred)
valid_x

,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
400,0.541169,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
401,0.533416,4,2,1.0,94.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
402,0.539708,5,2,2.0,88.0,0.0,45.0,10.0,0.0,54.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
403,0.532134,5,2,2.0,93.0,0.0,45.0,10.0,0.0,53.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
404,0.495551,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,0.919561,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
594,0.267534,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.47,53.07,50.89,55.1,66.49,1.0,0.0,0.0,0.0,0.0
595,0.248942,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,1.0,0.0,0.0,0.0,0.0
596,-0.326478,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [22]:
DT_valid = DT.predict(valid_x)
DT_valid

array([2, 1, 2, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1,
       0, 1, 0, 1, 2, 0, 2, 0, 1, 1, 1, 0, 1, 0, 0, 2, 1, 0, 2, 1, 0, 1,
       1, 1, 1, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 1, 1, 2, 2, 1, 1, 1, 2,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 2, 0, 1, 0, 1, 2, 0, 2, 0, 1, 2,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0,
       0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2])

In [24]:
DT_f1 = f1_score(DT_valid_y, DT_valid, average='macro')
DT_f1

0.34581768974272653